In [3]:
import pandas as pd
import random
from faker import Faker
import psycopg2

In [20]:

conn = psycopg2.connect(
    dbname="original",
    user="postgres",
    password="prashanta",
    host="localhost",
    port="5432"
)
# SQL query to fetch data
query = "SELECT * FROM accounts;"

# Load data into a pandas DataFrame
df = pd.read_sql_query(query, conn)


# Close the connection
conn.close()

# Display the DataFrame
df.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\100642893.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,_airbyte_ab_id,_airbyte_emitted_at,cin,fax,pan,ssn,url,zip,city,flg1,...,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,_airbyte_additional_properties,source_file_path,updated_at
0,273b5728-535b-4106-8c31-ce5e48eee381,2024-08-05 07:02:37.777,None,None,NaN,None,None,977000,None,None,...,None,None,None,None,None,None,None,None,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
1,70ced3d3-ad57-4433-a382-6ecbcc42fd90,2024-08-05 07:02:37.777,None,None,NaN,None,None,0,KATHM,None,...,None,None,None,None,None,None,None,None,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
2,15e20dca-a337-44b5-b202-b299895336df,2024-08-05 07:02:37.777,None,None,NaN,None,None,977000,BUDAN,None,...,None,None,None,None,None,None,None,None,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
3,5cf04c55-8085-4e86-ab6f-88904ceb2c6e,2024-08-05 07:02:37.777,None,None,NaN,None,None,977000,RAMNA,None,...,None,None,None,None,None,None,None,None,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
4,8fb8b8e6-aa95-4344-bdbe-a49935add064,2024-08-05 07:02:37.777,None,None,104894561.0,None,None,12,BIRAT,None,...,None,None,None,None,None,None,None,None,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596


In [21]:
df = df.drop(['_airbyte_ab_id', '_airbyte_emitted_at',"source_file_path","_airbyte_additional_properties"], axis=1)
df.head()

,cin,fax,pan,ssn,url,zip,city,flg1,flg2,flg3,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,None,None,NaN,None,None,977000,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,None,None,NaN,None,None,0,KATHM,None,None,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,None,None,NaN,None,None,977000,BUDAN,None,None,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,None,None,NaN,None,None,977000,RAMNA,None,None,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,None,None,104894561.0,None,None,12,BIRAT,None,None,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [22]:
#cust_type generation
import numpy as np
def get_cust_type(df):
    df= df.copy()
    condition =  (df["constitution_code"] == "INDIJ") | (df["constitution_code"] == "INDIV")
    df['cust_type'] = np.where(condition,"INDIVIDUAL","LEGAL")
    return df



df = get_cust_type(df)
df[["cust_type","constitution_code"]].head(7)

,cust_type,constitution_code
0,INDIVIDUAL,INDIV
1,LEGAL,OTHER
2,INDIVIDUAL,INDIV
3,INDIVIDUAL,INDIJ
4,INDIVIDUAL,INDIV
5,INDIVIDUAL,INDIV
6,LEGAL,USTS


In [23]:
# CIF_ID Generation
import pandas as pd
import numpy as np

def generate_cif_id(code):
    if code == "INDIVIDUAL":
        prefix = "DLNIND"
    elif code == "LEGAL":
        prefix = "DLNLEG"
    else:
        return ""  
    
    random_digits = str(np.random.randint(1000000, 9999999))
    return prefix + random_digits


df["cif_id"] = df["cust_type"].apply(generate_cif_id)
df.head()

,cin,fax,pan,ssn,url,zip,city,flg1,flg2,flg3,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,cif_id
0,None,None,NaN,None,None,977000,None,None,None,None,...,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596,DLNIND7738127
1,None,None,NaN,None,None,0,KATHM,None,None,None,...,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596,DLNLEG4776030
2,None,None,NaN,None,None,977000,BUDAN,None,None,None,...,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596,DLNIND4919530
3,None,None,NaN,None,None,977000,RAMNA,None,None,None,...,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596,DLNIND5183310
4,None,None,104894561.0,None,None,12,BIRAT,None,None,None,...,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596,DLNIND5881063


In [8]:
df["cif_id"].nunique()

125

In [24]:
import numpy as np
import random


df["marital_status"] = np.where(
    df["cust_type"] == "INDIVIDUAL",
    np.random.choice([True, False],size=len(df), p=[0.62, 0.38]),
    ""
)
df[["cust_type","marital_status"]].head(5)

,cust_type,marital_status
0,INDIVIDUAL,False
1,LEGAL,
2,INDIVIDUAL,True
3,INDIVIDUAL,False
4,INDIVIDUAL,True


In [25]:
selected_columns = [
    "cif_id",
    "orgkey",
    "cust_first_name",
    "cust_middle_name",
    "cust_last_name",
    "cust_dob",
    "cust_type",
    "gender",
    "marital_status",
    "education",
    "pan",
    "email",
    "preferredphone"
]
remaining_columns = [col for col in df.columns if col not in selected_columns]


In [26]:
new_df = df[selected_columns + remaining_columns]
new_df.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND7738127,319749297,SHYAM,None,CHAUDHARY,1988-02-17T00:00:00.000000,INDIVIDUAL,F,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,DLNLEG4776030,R24249553,None,None,NIC BANK-LOAN PROCESSING FEE,1900-01-01T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND4919530,R28893337,PRADYUMNA,LAL,RAJBAMSHI,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,DLNIND5183310,R66368828,CHANDESHWAR,SAHUTELI/SHAMBHUKUMAR,SAH,1965-07-16T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND5881063,R02063811,JAGADISH,PRASAD,AGRAWAL,1943-04-05T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [39]:
cust_Ind = new_df[new_df["cust_type"]=="INDIVIDUAL"]
cust_Ind.head(80)

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND7738127,319749297,SHYAM,None,CHAUDHARY,1988-02-17T00:00:00.000000,INDIVIDUAL,F,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND4919530,R28893337,PRADYUMNA,LAL,RAJBAMSHI,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,DLNIND5183310,R66368828,CHANDESHWAR,SAHUTELI/SHAMBHUKUMAR,SAH,1965-07-16T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND5881063,R02063811,JAGADISH,PRASAD,AGRAWAL,1943-04-05T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
5,DLNIND4552657,R00061514,BIRENDRA BAHADUR,None,NEUPANE,1978-06-13T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,DLNIND6433883,R01911511,TIKA,None,KUMARI PADHYA,1959-01-27T00:00:00.000000,INDIVIDUAL,F,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
121,DLNIND3175998,R01911516,BHARATI,None,KUMARI KHATRI,1997-09-15T00:00:00.000000,INDIVIDUAL,F,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
122,DLNIND1621279,R01911519,BASANTI,None,DAMAI,1968-04-22T00:00:00.000000,INDIVIDUAL,F,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
123,DLNIND5839511,R01911512,BUDAI,None,SAH,1971-05-05T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [31]:
print(len(cust_Ind))

71


In [34]:
df = pd.read_csv("shuffled_data.csv")
df = df.iloc[0:71]

In [38]:
print(len(df))

71


In [36]:
df

,cust_first_name,cust_middle_name,cust_last_name,gender,cust_type
0,Buddhiraj,Bishal,Sharma,M,INDIVIDUAL
1,Pramil,Krishna,Neupane,M,INDIVIDUAL
2,Shirish,Gopal,Thapa,M,INDIVIDUAL
3,Hina,Devi,Bhattarai,F,INDIVIDUAL
4,Poshan,Bahadur,Thapa,M,INDIVIDUAL
...,...,...,...,...,...
66,Heli,Saraswati,Timalsina,F,INDIVIDUAL
67,Pranij,Bibek,Shrestha,M,INDIVIDUAL
68,Sanjita,Kumar,Pandey,M,INDIVIDUAL
69,Prajina,Kumari,Pandey,F,INDIVIDUAL


In [ ]:
cust_Ind["cust_first_name"] = df["cust_first_name"].values
cust_Ind["name"] = df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
cust_Ind["preferredname"] =  df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
cust_Ind["cust_middle_name"] = df["cust_middle_name"].values
cust_Ind["cust_last_name"] = df["cust_last_name"].values
cust_Ind["gender"] = df["gender"].values
cust_Ind["short_name"] = df["cust_first_name"].values
cust_Ind.head(80)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\3822427701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_Ind["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\3822427701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_Ind["name"] = df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\3822427701.py:3: SettingWithCopyWarni

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND7738127,319749297,Buddhiraj,Bishal,Sharma,1988-02-17T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND4919530,R28893337,Pramil,Krishna,Neupane,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,DLNIND5183310,R66368828,Shirish,Gopal,Thapa,1965-07-16T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND5881063,R02063811,Hina,Devi,Bhattarai,1943-04-05T00:00:00.000000,INDIVIDUAL,F,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
5,DLNIND4552657,R00061514,Poshan,Bahadur,Thapa,1978-06-13T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,DLNIND6433883,R01911511,Heli,Saraswati,Timalsina,1959-01-27T00:00:00.000000,INDIVIDUAL,F,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
121,DLNIND3175998,R01911516,Pranij,Bibek,Shrestha,1997-09-15T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
122,DLNIND1621279,R01911519,Sanjita,Kumar,Pandey,1968-04-22T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
123,DLNIND5839511,R01911512,Prajina,Kumari,Pandey,1971-05-05T00:00:00.000000,INDIVIDUAL,F,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [44]:
df = pd.read_csv("company_name.csv")
df = df.iloc[0:54]
df.head()

,cust_first_name,cust_type
0,TANAHUN DIGITAL PVT. LTD.,LEGAL
1,PYUTHAN LOGISTICS PVT. LTD.,LEGAL
2,PATHIVARA LOGISTICS PVT. LTD.,LEGAL
3,DLYTICA ACADEMY,LEGAL
4,PARSA TRADING PVT. LTD.,LEGAL


In [48]:
print(len(df))

54


In [47]:
cust_leg = new_df[new_df["cust_type"]=="LEGAL"]
cust_leg.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
1,DLNLEG4776030,R24249553,None,None,NIC BANK-LOAN PROCESSING FEE,1900-01-01T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
6,DLNLEG5895248,C00004434,None,None,BARPIPAL TOLE BATO UPAVAOKTA SAMITI,2019-01-22T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
7,DLNLEG1292230,R08272460,None,None,CRYSTAL OFFSET PRESS,1900-01-01T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
8,DLNLEG5379725,C00004439,None,None,ARJUN SUNCHANDI UDHYOG,2015-09-15T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
9,DLNLEG7509007,C00004426,None,None,ROYAL FURNITURE UDYOG,2019-01-31T00:00:00.000000,LEGAL,None,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [49]:
print(len(cust_leg))

54


In [ ]:
cust_leg["cust_first_name"] = df["cust_first_name"].values
cust_leg["name"] = df["cust_first_name"].values
cust_leg["preferredname"] = df["cust_first_name"]
cust_leg["cust_middle_name"] = ""
cust_leg["cust_last_name"] = ""
cust_leg["gender"] = ""
cust_leg["short_name"] = df["cust_first_name"].values
cust_leg.head(5)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\1676386299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_leg["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\1676386299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_leg["name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\1676386299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
1,DLNLEG4776030,R24249553,TANAHUN DIGITAL PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
6,DLNLEG5895248,C00004434,PYUTHAN LOGISTICS PVT. LTD.,,,2019-01-22T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
7,DLNLEG1292230,R08272460,PATHIVARA LOGISTICS PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
8,DLNLEG5379725,C00004439,DLYTICA ACADEMY,,,2015-09-15T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
9,DLNLEG7509007,C00004426,PARSA TRADING PVT. LTD.,,,2019-01-31T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [51]:
result = pd.concat([cust_Ind,cust_leg])
shuffled = result.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND1405573,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,DLNIND2302056,R01651971,Prithivi,Kumar,Yadav,1993-02-28T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND3356902,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,DLNLEG7954119,R22132679,MANANG HOSPITALITY PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND7909079,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [52]:
import numpy as np

def generate_email(data):
    # Extract first name (handle NaN and company names)
    first_name = str(data["cust_first_name"]) if not pd.isna(data["cust_first_name"]) else "unknown"
    
    # Remove "PVT. LTD." or "LTD." and clean the name
    name = (
        first_name.replace("PVT. LTD.", "")
                 .replace("LTD.", "")
                 .strip() 
                 .lower()  
                 .replace(" ", "")
    )
    
    num = str(np.random.randint(100, 999))
    
    if data["cust_type"] == "INDIVIDUAL":
        email = f"{name}{num}@gmail.com"
    else:
        email = f"{name}{num}@outlook.com"
    
    return email

# Apply the function to the DataFrame
shuffled["email"] = shuffled[["cust_first_name", "cust_type"]].apply(generate_email, axis=1)

# Display first 15 generated emails
shuffled["email"].head(15)

0                   suprim857@gmail.com
1                 prithivi853@gmail.com
2                   nawraj282@gmail.com
3      mananghospitality737@outlook.com
4                shrikumar254@gmail.com
5       tamorelectronics713@outlook.com
6                   hansha727@gmail.com
7          nepalairlines402@outlook.com
8     sarlahiengineering419@outlook.com
9                   poshan894@gmail.com
10                  aavaas624@gmail.com
11        bardiabuilders951@outlook.com
12                  sakhil814@gmail.com
13          khotangfoods429@outlook.com
14               anushikha828@gmail.com
Name: email, dtype: object

In [53]:
shuffled.head(99)

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND1405573,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,DLNIND2302056,R01651971,Prithivi,Kumar,Yadav,1993-02-28T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND3356902,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
3,DLNLEG7954119,R22132679,MANANG HOSPITALITY PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND7909079,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,DLNIND4970185,R01786441,Keith,Bhakta,Rai,2001-08-19T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
95,DLNLEG7509007,C00004426,PARSA TRADING PVT. LTD.,,,2019-01-31T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
96,DLNIND2522297,R01009724,Mikma,Biren,Thapa,1982-06-01T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
97,DLNLEG8053657,C00012403,PHEWA AUTO PARTS PVT. LTD.,,,2014-04-09T00:00:00.000000,LEGAL,,,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [54]:
def generate_phone_number(data):
   return  "+977-98"+str(np.random.randint(10000000,99999999))

shuffled["preferredphone"] = shuffled["preferredphone"].apply(generate_phone_number)
shuffled["preferredphone"].head(10)

0    +977-9876533295
1    +977-9883024611
2    +977-9876412482
3    +977-9830440306
4    +977-9836396986
5    +977-9866263368
6    +977-9838359478
7    +977-9894986950
8    +977-9863184824
9    +977-9826056824
Name: preferredphone, dtype: object

In [66]:
Ind_cust = shuffled[shuffled["cust_type"] == "INDIVIDUAL"]
print(len(Ind_cust))

71


In [67]:
first_5 = Ind_cust.iloc[:5]        
remaining = Ind_cust.iloc[5:] 

In [68]:
first_5.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND1405573,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,DLNIND2302056,R01651971,Prithivi,Kumar,Yadav,1993-02-28T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND3356902,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND7909079,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
6,DLNIND1682141,019898447,Hansha,Bibek,Adhikari,1991-06-03T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [69]:
import pandas as pd
import numpy as np

# Create sample data
data = {
    'cust_first_name': ['Prashanta', 'Kabita', 'sandal', 'sirish', 'Veshraj'],
    'cust_middle_name': ['Babu', '', '', '', ''],
    'cust_last_name': ['Thapa', 'Bhattarai', 'Tandukar', 'poudel', 'Mainali'],
    'preferredphone': ['9810288411', '9866180896', '9861396794', '9861632577', '9861674997'],
    'email': ['Prashant.babu@dlytica.com', 'kabita.bhattarai@dlytica.com', 'sandil.tandukar@dlytica.com', 'sirish.poudel@dlytica.com', 'Veshraj.Mainali@dlytica.com'],
    'preferredemail': ['john.smith@example.com', 'emily.johnson@example.com', 'michael.williams@example.com', 'sarah.brown@example.com', 'david.jones@example.com']
}

# Create DataFrame
df = pd.DataFrame(data)

df.head()

,cust_first_name,cust_middle_name,cust_last_name,preferredphone,email,preferredemail
0,Prashanta,Babu,Thapa,9810288411,Prashant.babu@dlytica.com,john.smith@example.com
1,Kabita,,Bhattarai,9866180896,kabita.bhattarai@dlytica.com,emily.johnson@example.com
2,sandal,,Tandukar,9861396794,sandil.tandukar@dlytica.com,michael.williams@example.com
3,sirish,,poudel,9861632577,sirish.poudel@dlytica.com,sarah.brown@example.com
4,Veshraj,,Mainali,9861674997,Veshraj.Mainali@dlytica.com,david.jones@example.com


In [70]:
first_5["cust_first_name"] = df["cust_first_name"].values
first_5["cust_middle_name"] = df["cust_middle_name"].values
first_5["cust_last_name"] = df["cust_last_name"].values
first_5["preferredphone"] = df["preferredphone"].values
first_5["email"] = df["email"].values
first_5["preferredemail"] = df["email"].values
first_5["name"] = df["cust_first_name"].values + " " + df["cust_middle_name"].values + " " + df["cust_last_name"].values
first_5["preferredname"] = df["cust_first_name"].values + " " + df["cust_middle_name"].values + " " + df["cust_last_name"].values
first_5.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\103564471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_5["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\103564471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_5["cust_middle_name"] = df["cust_middle_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11184\103564471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,DLNIND1405573,R01516157,Prashanta,Babu,Thapa,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
1,DLNIND2302056,R01651971,Kabita,,Bhattarai,1993-02-28T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
2,DLNIND3356902,R09396490,sandal,,Tandukar,2013-01-09T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
4,DLNIND7909079,R00580783,sirish,,poudel,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596
6,DLNIND1682141,019898447,Veshraj,,Mainali,1991-06-03T00:00:00.000000,INDIVIDUAL,M,True,None,...,None,None,None,None,None,None,None,None,None,2024-08-07 06:29:18.596


In [71]:
leg_cust = shuffled[shuffled["cust_type"] == "LEGAL"]
print(len(Ind_cust))

71


In [ ]:
result = pd.concat([first_5,remaining,leg_cust])
shuffled = result.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
leg_cust = shuffled[shuffled["cust_type"] == "LEGAL"]

In [ ]:
import pandas as pd
import numpy as np

# Create sample data
data = {
    'cust_first_name': ['Prashanta', 'Kabita', 'sandal', 'sirish', 'Veshraj'],
    'cust_middle_name': ['Babu', '', '', '', ''],
    'cust_last_name': ['Thapa', 'Bhattarai', 'Tandukar', 'poudel', 'Mainali'],
    'preferredphone': ['9810288411', '9866180896', '9861396794', '9861632577', '9861674997'],
    'email': ['Prashant.babu@dlytica.com', 'kabita.bhattarai@dlytica.com', 'sandil.tandukar@dlytica.com', 'sirish.poudel@dlytica.com', 'Veshraj.Mainali@dlytica.com'],
    'preferredemail': ['john.smith@example.com', 'emily.johnson@example.com', 'michael.williams@example.com', 'sarah.brown@example.com', 'david.jones@example.com']
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [ ]:
shuffled.to_csv('account_masked.csv', index=False) 